<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/P1-NL2SQL/6_2_Azure_NL2SQL_Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>6.2-Calling Azure OpenAI Services from a Notebook.</h2>
    <h3></h3>
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)
___________
Models:

Colab Environment: CPU

Keys:
* Azure
* NL2SQL
___________


In this notebook, we make a call to a GPT3.5 model that we've set up from Azure OpenAI Studio to function as a translator for SQL queries from natural language.

The prompt structure has been created and explained in a [previous article](https://colab.research.google.com/drive/1mbIAv1p6nqtsngXfCG5WrozlvmoJI06F#scrollTo=uxFVaxlskUf-&line=3&uniqifier=1), part of the Large Language Models course.

In [ ]:
#Install openai
!pip install -q openai==1.30.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
import os
import openai


You can Obtain the **prompt**  the **api_base** and the **key** from the Sample Code window on Chat Playground in Azure Open Studio.

![My Image](https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_6-13.jpg?raw=true)


In [ ]:
from getpass import getpass
os.environ["AZURE_OPENAI_KEY"] = getpass("Azure AI key:")

Azure AI key:··········


In [ ]:

client = AzureOpenAI(
  azure_endpoint = "https://largelanguagemodelsprojects.openai.azure.com/",
  api_key=os.getenv("AZURE_OPENAI_KEY"),
  api_version="2024-02-15-preview"
)


In [ ]:
context = [{"role":"system","content":"create table employees( \n        ID_Usr INT primary key,\n        name VARCHAR);\n    /*3 example rows\n    select * from employees limit 3;\n    ID_Usr    name\n    1344      George StPierre\n    2122      Jon jones\n    1265      Anderson Silva\n    */\n\n    create table salary(\n        ID_Usr INT,\n        year DATE,\n        salary FLOAT,\n        foreign key (ID_Usr) references employees(ID_Usr));\n    /*3 example rows\n    select * from salary limit 3\n    ID_Usr    date          salary\n    1344      01/01/2023    61000\n    1344      01/01/2022    60000\n    1265      01/01/2023    55000\n    */\n\n    create table studies(\n        ID_study INT,\n        ID_Usr INT,\n        educational_level INT,  /* 5=phd, 4=Master, 3=Bachelor */\n        Institution VARCHAR,\n        Years DATE,\n        Speciality VARCHAR,\n        primary key (ID_study, ID_Usr),\n        foreign key(ID_Usr) references employees (ID_Usr));\n    /*3 example rows\n    select * from studies limit 3\n    ID_Study ID_Usr educational_level Institution    Years       Speciality\n    2782     1344   3                 UC San Diego   01/01/2010  Bachelor of Science in Marketing\n    2334     1344   5                 MIT            01/01/2023  Phd. Data Science.\n    2782     2122   3                 UC San Diego   01/01/2010  Bachelor of Science in Marketing\n    */\n-- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions, returning only SQL code, for the table provided above."},{"role":"user","content":"How Many employes we have with a salary bigger than 50000?"},{"role":"assistant","content":"SELECT COUNT(*) AS total_employees\nFROM employees e\nINNER JOIN salary s ON e.ID_Usr = s.ID_Usr\nWHERE s.salary > 50000;"},{"role":"user","content":"Return the names of the three people who have had the highest salary increase in the last three years."},{"role":"assistant","content":"SELECT e.name\nFROM employees e\nJOIN salary s ON e.ID_usr = s.ID_usr\nWHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)\nGROUP BY e.name\nORDER BY (MAX(s.salary) - MIN(s.salary)) DESC\nLIMIT 3;"},{"role":"user","content":"Give me the name of the worst paid emploee"},{"role":"assistant","content":"SELECT e.name\nFROM employees e\nJOIN salary s ON e.ID_usr = s.ID_usr\nWHERE s.salary = (SELECT MIN(salary) FROM salary);"}]

In [ ]:
context = [{"role":"system","content":"""
            create table employees(
              ID_Usr INT primary key,
              name VARCHAR);
              /*3 example rows
              select * from employees limit 3;
              ID_Usr    name
              1344      George StPierre
              2122      Jon jones
              1265      Anderson Silva
              */

            create table salary(
              ID_Usr INT,
              year DATE,
              salary FLOAT,
              foreign key (ID_Usr) references employees(ID_Usr));
              /*3 example rows
              select * from salary limit 3
              ID_Usr    date          salary
              1344      01/01/2023    61000
              1344      01/01/2022    60000
              1265      01/01/2023    55000
              */

            create table studies(
              ID_study INT,
              ID_Usr INT,
              educational_level INT,  /* 5=phd, 4=Master, 3=Bachelor */
              Institution VARCHAR,
              Years DATE,
              Speciality VARCHAR,
              primary key (ID_study, ID_Usr),
              foreign key(ID_Usr) references employees (ID_Usr));
              /*3 example rows
              select * from studies limit 3
              ID_Study ID_Usr educational_level Institution    Years       Speciality
              2782     1344   3                 UC San Diego   01/01/2010  Bachelor of Science in Marketing
              2334     1344   5                 MIT            01/01/2023  Phd. Data Science.
              2782     2122   3                 UC San Diego   01/01/2010  Bachelor of Science in Marketing
              */
              -- Maintain the SQL order simple and efficient as you can,
              using valid SQL Lite, answer the following questions,
              returning only SQL code, for the table provided above."""},
            {"role":"user","content":"""How Many employes we have with a salary bigger than 50000?"""},
            {"role":"assistant","content":"""
            SELECT COUNT(*) AS total_employees
            FROM employees e
            INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
            WHERE s.salary > 50000;"""},
            {"role":"user","content":"Return the names of the three people who have had the highest salary increase in the last three years."},
            {"role":"assistant","content":"""
            SELECT e.name
            FROM employees e
            JOIN salary s ON e.ID_usr = s.ID_usr
            WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
            GROUP BY e.name
            ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC\nLIMIT 3;"""}]

In [ ]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
      model="GPT35NL2SQL", #Our deployment
      messages = newcontext,
      temperature=0,
      max_tokens=800)

    return (response.choices[0].message.content)

In [ ]:
context_user = context.copy()
response = return_CCRMSQL("What's the name of the best paid employee?", context_user)
print(response)


            SELECT e.name
            FROM employees e
            JOIN salary s ON e.ID_usr = s.ID_usr
            WHERE s.salary = (SELECT MAX(salary) FROM salary)
            LIMIT 1;


In [ ]:
print(return_CCRMSQL("Return the Institution with a higher average salary", context_user))

SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_Usr = e.ID_Usr
JOIN salary sa ON e.ID_Usr = sa.ID_Usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;


#Conclusions.
Once you have the prompt to use, set up the OpenAI services in Azure is straightforward.

I observed some differences in the SQL from the one obtained calling directly to OpenAI API.
